In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

from resnet import ResNet20

import os
import torch
import torch.optim as optim
import torch.nn as nn
from datetime import datetime

from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
hyperparameters = {
    'sparsity_type': "entropy", # nm / entropy / feather / spartan / ses / base (no sparsity)
    'epochs': 200,
    'lr': 0.1,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'batch_size': 128,
}

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.201]),
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.201]),
])


train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size'], shuffle=False, num_workers=2)

In [ ]:
class Pruner:
    def __init__(self, model):
        self.entropy_threshold = 
        self.model = model

    def calculate_entropy(self, weight: torch.Tensor, eps=1e-10):
        flattened = weight.abs().flatten()
        probs = flattened / torch.sum(flattened)
        entropy = - torch.sum(probs * torch.log(probs + eps))
        return entropy

    def apply_entropy_sparsity(self):
        for name, param in self.model.named_parameters():
            if "weight" in name:
                entropy = self.calculate_entropy(param)
                if entropy < self.entropy_threshold:
                    param.data.zero_()

    def print_sparsity(self):
        tot_params, tot_zeros = 0, 0
        for name, module in self.model.named_modules():
            if isinstance(module, nn.Conv2d):
                weight = module.weight.data
                n_params = weight.numel()
                n_zeros = torch.sum(weight == 0).item()

                tot_params += n_params
                tot_zeros += n_zeros

                print(f"{name}: Total Params = {n_params}. Total zero params = {n_zeros}. Sparsity = {n_zeros / n_params:.2%}")
            

In [ ]:
def train(model, train_loader, criterion, optimizer, epoch, log_file):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}", ncols=100)
    
    for batch_idx, (inputs, targets) in enumerate(pbar):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

        pbar.set_postfix(loss=running_loss/(batch_idx+1), accuracy=100.0 * correct / total)

    avg_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct / total
    log_file.write(f'Epoch [{epoch+1}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

    return avg_loss, accuracy


In [ ]:
def test(model, test_loader, criterion, log_file):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    pbar = tqdm(test_loader, desc="Testing", ncols=100)
    
    with torch.no_grad():
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)

            pbar.set_postfix(loss=test_loss/(total + inputs.size(0)), accuracy=100.0 * correct / total)

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100.0 * correct / total
    log_file.write(f'Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

    return avg_test_loss, accuracy

In [ ]:
resnet20_model = ResNet20()
resnet20_model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet20_model.parameters(), lr=hyperparameters['lr'], 
                      momentum=hyperparameters['momentum'], weight_decay=hyperparameters['weight_decay'])

In [ ]:
current_learning_rate = hyperparameters['lr']
DECAY = 0.1
DECAY_EPOCHS = [100, 150]
EPOCHS = 200

In [ ]:
sparsity_type = hyperparameters['sparsity_type']
output_base_path = "/home/sg666/Class/ECE661/outputs"
sparsity_folder_path = os.path.join(output_base_path, sparsity_type)

hyperparameter_str = f"epochs_{hyperparameters['epochs']}_lr_{hyperparameters['lr']}_momentum_{hyperparameters['momentum']}_wd_{hyperparameters['weight_decay']}_batch_{hyperparameters['batch_size']}"
output_folder = os.path.join(sparsity_folder_path, hyperparameter_str)

os.makedirs(output_folder, exist_ok=True)

hyperparameter_file = os.path.join(output_folder, 'hyperparameters.txt')
with open(hyperparameter_file, 'w') as f:
    for key, value in hyperparameters.items():
        f.write(f"{key}: {value}\n")

log_file_path = os.path.join(output_folder, 'training_log.txt')

In [ ]:
pruner = Pruner(resnet20_model, 

In [ ]:
with open(log_file_path, 'w') as log_file:
    log_file.write(f"Training started at {datetime.now()}\n")

    best_accuracy = 0.0
    
    for epoch in range(hyperparameters['epochs']):
        # Apply entropy sparsity at the start of every epoch
        pruner.apply_entropy_sparsity()

        train_loss, train_accuracy = train(resnet20_model, train_loader, criterion, optimizer, epoch, log_file)
        test_loss, test_accuracy = test(resnet20_model, test_loader, criterion, log_file)

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            model_checkpoint_path = os.path.join(output_folder, f"best_model.pth")
            torch.save(resnet20_model.state_dict(), model_checkpoint_path)
            print(f"Saved best model at epoch {epoch+1} with accuracy: {best_accuracy:.2f}%")


        if epoch+1 in DECAY_EPOCHS:
            current_learning_rate = current_learning_rate * DECAY
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_learning_rate
            print("Current learning rate has decayed to %f" %current_learning_rate)

    log_file.write(f"Training completed at {datetime.now()}\n")
    log_file.write(f"Best model accuracy: {best_accuracy:.2f}%\n")
